In [1]:
import os
import time


def add_1(data, idx, max_key, cnt):
    data[0] = [[idx, cnt]] + data[0]
    current = 0
    while len(data[current]) > 2:
        assert len(data[current]) == 3
        buckets = data[current][-2:]
        data[current] = data[current][:1]
        current += 1
        
        if data.get(current, None) is None:
            data[current] = [buckets[0]]
            max_key = current
            # assert max_key == current
        else:
            data[current] = [buckets[0]] + data[current]
    return max_key


def output_data(data, max_key):
    cnt = 0
    mul = 1
    for i in range(max_key + 1):
        # print(len(data[i]) * mul)
        cnt += len(data[i]) * mul
        mul *= 2
    print("Count of 1's:", cnt - mul // 4)
    return cnt - mul // 4


cnt = 0
idx = 0
bucket_dict = {0: []}
max_key = 0
window_size = 1000

start = time.time()
with open("stream_data_dgim.txt", "r") as f:
    while True:
        input_bit = f.read(1)
        if not input_bit:
            DGIM_count = output_data(bucket_dict, max_key)
            break
        if input_bit == "\t":
            continue
        if len(bucket_dict[0]) > 0 and idx == bucket_dict[max_key][-1][0]:
            bucket_dict[max_key] = bucket_dict[max_key][:-1]
            if len(bucket_dict[max_key]) == 0:
                bucket_dict.pop(max_key)
                max_key -= 1
        if input_bit == '1':
            max_key = add_1(bucket_dict, idx, max_key, cnt)
        cnt += 1
        idx = cnt % window_size
        
end = time.time()
print("Run Time(s): %.4f" % (end - start))

Count of 1's: 508
Run Time(s): 0.0614


In [3]:
def get_last_k(data, k, last, max_key):
    target = last - k
    res = 0
    mul = 1
    for i in range(max_key + 1):
        for j in range(len(data[i])):
            if data[i][j][1] > target:
                res += mul
            else:
                if j == 0:
                    res -= mul // 4
                else:
                    res -= mul // 2
                return res
        mul *= 2

print("count of 1's in the last 200 bits:", get_last_k(bucket_dict, 200, cnt, max_key))
print("count of 1's in the last 500 bits:", get_last_k(bucket_dict, 500, cnt, max_key))

count of 1's in the last 200 bits: 76
count of 1's in the last 500 bits: 220


In [4]:
window = []
counter = 0

start = time.time()
with open("stream_data_dgim.txt", "r") as f:
    while True:
        input_bit = f.read(1)
        if not input_bit:
            print("Accurate Count:", counter)
            break
        if input_bit not in ['0', '1']:
            continue
        if len(window) < window_size:
            window.append(input_bit)
            if input_bit == '1':
                counter += 1
        else:
            out_bit = window[0]
            window = window[1:] + [input_bit]
            if input_bit == '1' and out_bit != '1':
                counter += 1
            elif input_bit != '1' and out_bit == '1':
                counter -= 1
end = time.time()
print("Accurate Count Run Time(s): %.4f" % (end - start))
print("Error: %.2f" % (abs(DGIM_count - counter) * 100 / counter), "%")

Accurate Count: 391
Accurate Count Run Time(s): 0.2518
Error: 29.92 %
